In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

L, W = 1.0, 0.5  # Length and width of the domain (in cm)
lambda_ = 5.0e9  # Elastic constant (Pa)
mu = 5.0e9       # Shear modulus (Pa)
h = 1.0          # Thickness (cm)

class PINN(nn.Module):
    def __init__(self):
        super(PINN, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(2, 128),
            nn.Tanh(),
            nn.Linear(128, 128),
            nn.Tanh(),
            nn.Linear(128, 128),
            nn.Tanh(),
            nn.Linear(128, 2)
        )

    def forward(self, x, y):
        inputs = torch.cat([x, y], dim=1)
        raw = self.net(inputs)          

    # Force displacement to be zero at x=-0.5:
    # (x+0.5) is zero on the left boundary
        enforced = torch.zeros_like(raw)
        enforced[:, 0] = (x.flatten() + 0.5) * raw[:, 0]
        enforced[:, 1] = (x.flatten() + 0.5) * raw[:, 1]
        return enforced


def strain_tensor(u_x, u_y, x, y):
    u_x_x = torch.autograd.grad(u_x, x, grad_outputs=torch.ones_like(u_x), retain_graph=True, create_graph=True)[0]
    u_y_y = torch.autograd.grad(u_y, y, grad_outputs=torch.ones_like(u_y), retain_graph=True, create_graph=True, allow_unused=True)[0]
    u_x_y = torch.autograd.grad(u_x, y, grad_outputs=torch.ones_like(u_x), retain_graph=True, create_graph=True)[0]
    u_y_x = torch.autograd.grad(u_y, x, grad_outputs=torch.ones_like(u_y), retain_graph=True, create_graph=True)[0]

    E_xx = u_x_x
    E_yy = u_y_y
    E_xy = 0.5 * (u_x_y + u_y_x)
    return E_xx, E_yy, E_xy

def stress_tensor(E_xx, E_yy, E_xy):
    scale_factor = 1e9  # Scaling factor for material constants
    trace_E = E_xx + E_yy
    sigma_xx = h * ((lambda_ / scale_factor) * trace_E + 2 * (mu / scale_factor) * E_xx)
    sigma_yy = h * ((lambda_ / scale_factor) * trace_E + 2 * (mu / scale_factor) * E_yy)
    sigma_xy = h * (2 * (mu / scale_factor) * E_xy)
    return sigma_xx, sigma_yy, sigma_xy

def physics_loss(model, x, y):
    x.requires_grad_(True)
    y.requires_grad_(True)
    u = model(x, y)
    u_x, u_y = u[:, 0:1], u[:, 1:2]

    E_xx, E_yy, E_xy = strain_tensor(u_x, u_y, x, y)
    sigma_xx, sigma_yy, sigma_xy = stress_tensor(E_xx, E_yy, E_xy)

    sigma_xx_x = torch.autograd.grad(sigma_xx, x, grad_outputs=torch.ones_like(sigma_xx), retain_graph=True, create_graph=True)[0]
    sigma_xy_y = torch.autograd.grad(sigma_xy, y, grad_outputs=torch.ones_like(sigma_xy), retain_graph=True, create_graph=True)[0]
    sigma_yy_y = torch.autograd.grad(sigma_yy, y, grad_outputs=torch.ones_like(sigma_yy), retain_graph=True, create_graph=True)[0]
    sigma_xy_x = torch.autograd.grad(sigma_xy, x, grad_outputs=torch.ones_like(sigma_xy), retain_graph=True, create_graph=True)[0]

    residual_x = sigma_xx_x + sigma_xy_y
    residual_y = sigma_yy_y + sigma_xy_x

    loss_equilibrium = torch.mean(residual_x**2 + residual_y**2)
    return loss_equilibrium

def boundary_condition_loss(model, L, W):
    # Boundary A: u_x = 0, u_y = 0 at x = -L/2
    y_A = torch.linspace(-W / 2, W / 2, 100).reshape(-1, 1).requires_grad_()
    x_A = -L / 2 * torch.ones_like(y_A, requires_grad=True)
    u_A = model(x_A, y_A)
    loss_A = torch.mean(u_A**2)

    # Boundary D: u_x = 0.025 * L, u_y = 0 at x = L/2
    y_D = torch.linspace(-W / 2, W / 2, 100).reshape(-1, 1).requires_grad_()
    x_D =  L / 2 * torch.ones_like(y_D, requires_grad=True)
    u_D = model(x_D, y_D)
    loss_D = torch.mean((u_D[:, 1:2])**2 + (u_D[:, 0:1] - 0.025 * L)**2)

    # Boundary B: traction-free (sigma_xx = sigma_xy = 0) at y = W/2
    x_B = torch.linspace(-L / 2, L / 2, 100).reshape(-1, 1).requires_grad_()
    y_B =  W / 2 * torch.ones_like(x_B, requires_grad=True)
    u_B = model(x_B, y_B)
    u_B_x, u_B_y = u_B[:, 0:1], u_B[:, 1:2]
    E_xx_B, E_yy_B, E_xy_B = strain_tensor(u_B_x, u_B_y, x_B, y_B)
    sigma_xx_B, sigma_yy_B, sigma_xy_B = stress_tensor(E_xx_B, E_yy_B, E_xy_B)
    loss_B = torch.mean(sigma_yy_B**2 + sigma_xy_B**2)

    # Boundary C: traction-free (sigma_xx = sigma_xy = 0) at y = -W/2
    x_C = torch.linspace(-L / 2, L / 2, 100).reshape(-1, 1).requires_grad_()
    y_C = -W / 2 * torch.ones_like(x_C, requires_grad=True)
    u_C = model(x_C, y_C)
    u_C_x, u_C_y = u_C[:, 0:1], u_C[:, 1:2]
    E_xx_C, E_yy_C, E_xy_C = strain_tensor(u_C_x, u_C_y, x_C, y_C)
    sigma_xx_C, sigma_yy_C, sigma_xy_C = stress_tensor(E_xx_C, E_yy_C, E_xy_C)
    loss_C = torch.mean(sigma_yy_C**2 + sigma_xy_C**2)

    # Force the model to have u_x=0, u_y=0 at x=-0.5, y=-0.25 specifically
    x_corner = torch.tensor([[-L/2]], dtype=torch.float32, requires_grad=True)
    y_corner = torch.tensor([[-W/2]], dtype=torch.float32, requires_grad=True)
    u_corner = model(x_corner, y_corner)

    # 2) Corner loss:
    loss_corner = torch.mean(u_corner**2)

    # 3) Multiply by a LARGE weight to force near-zero predictions:
    corner_weight = 1e9  # or 1e4, tune as needed
    return loss_A + loss_D + (loss_B / 1e18) + (loss_C / 1e18) # + corner_weight * loss_corner

def train_pinn(model, optimizer, n_epochs, n_points, L, W):
    loss_history = []

    for epoch in range(n_epochs):
        # Sample points inside the domain
        x = torch.rand((n_points, 1)) * L - L / 2
        y = torch.rand((n_points, 1)) * W - W / 2

        loss_pde = physics_loss(model, x, y)
        loss_bc = boundary_condition_loss(model, L, W)
        loss = loss_pde + loss_bc

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss_history.append(loss.item())
        if epoch % 500 == 0:
            print(f"Epoch {epoch}, Loss: {loss.item():.6f}")

    return loss_history


# Initialize model and optimizer
model = PINN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the model
n_epochs = 10000
n_points = 1000
loss_history = train_pinn(model, optimizer, n_epochs, n_points, L, W)

import pandas as pd
from sklearn.metrics import mean_squared_error 
file_path = '/Users/murat/Downloads/data.csv'  # Update this with the correct path
comparison_data = pd.read_csv(file_path)
x_values = torch.tensor(comparison_data['X'].values, dtype=torch.float32).reshape(-1, 1)
y_values = torch.tensor(comparison_data['Y'].values, dtype=torch.float32).reshape(-1, 1)
u_x_actual = comparison_data['u_x_actual'].values
u_y_actual = comparison_data['u_y_actual'].values

model.eval()
with torch.no_grad():
    u_pred = model(x_values, y_values) 
    u_x_pred = u_pred[:, 0].reshape(-1).numpy()
    u_y_pred = u_pred[:, 1].reshape(-1).numpy()

comparison_data['u_x_pred'] = u_x_pred
comparison_data['u_y_pred'] = u_y_pred

comparison_data['error_u_x'] = abs(comparison_data['u_x_actual'] - comparison_data['u_x_pred'])
comparison_data['error_u_y'] = abs(comparison_data['u_y_actual'] - comparison_data['u_y_pred'])
comparison_data['percent_error_u_x'] = abs(u_x_actual - u_x_pred) / abs(u_x_actual) * 100 
comparison_data['percent_error_u_y'] = abs(u_y_actual - u_y_pred) / abs(u_y_actual) * 100 

print(comparison_data)

Epoch 0, Loss: 0.807504
Epoch 500, Loss: 0.000018
Epoch 1000, Loss: 0.000012
Epoch 1500, Loss: 0.000010
Epoch 2000, Loss: 0.000007
Epoch 2500, Loss: 0.000005
Epoch 3000, Loss: 0.000004
Epoch 3500, Loss: 0.000003
Epoch 4000, Loss: 0.000003
Epoch 4500, Loss: 0.000013
Epoch 5000, Loss: 0.000002
Epoch 5500, Loss: 0.000002
Epoch 6000, Loss: 0.000001
Epoch 6500, Loss: 0.000018
Epoch 7000, Loss: 0.000760
Epoch 7500, Loss: 0.000036
Epoch 8000, Loss: 0.000019
Epoch 8500, Loss: 0.000118
